In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

C:\Users\Chris\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def data():
    # Split train & validation set
    DDD_Train_Decile10 = pd.read_pickle("DDD_Train_Decile10.pkl")
    DDD_Train_Targets_1D = pd.read_pickle("DDD_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("DDD_Test_Decile10.pkl")
    y_test = pd.read_pickle("DDD_Test_Targets_1D.pkl").astype(int)
    
    X_train, X_val, y_train, y_val = train_test_split(DDD_Train_Decile10, DDD_Train_Targets_1D, test_size=0.30, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,100,150,200])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM DDD Decile 10')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

1981/1981 [==============================] - 6s 3ms/step - loss: 0.3406 - acc: 0.8597 - val_loss: 0.2757 - val_acc: 0.9071
Epoch 39/100
1981/1981 [==============================] - 6s 3ms/step - loss: 0.3294 - acc: 0.8708 - val_loss: 0.2753 - val_acc: 0.9059
Epoch 40/100
1981/1981 [==============================] - 6s 3ms/step - loss: 0.3359 - acc: 0.8627 - val_loss: 0.2738 - val_acc: 0.9082
Epoch 41/100
1981/1981 [==============================] - 6s 3ms/step - loss: 0.3364 - acc: 0.8682 - val_loss: 0.2707 - val_acc: 0.9094
Epoch 42/100
1981/1981 [==============================] - 6s 3ms/step - loss: 0.3280 - acc: 0.8733 - val_loss: 0.2690 - val_acc: 0.8988
Epoch 43/100
1981/1981 [==============================] - 6s 3ms/step - loss: 0.3291 - acc: 0.8753 - val_loss: 0.2675 - val_acc: 0.9047
Epoch 44/100
1981/1981 [==============================] - 6s 3ms/step - loss: 0.3213 - acc: 0.8688 - val_loss: 0.2661 - val_acc: 0.9082
Epoch 45/100
1981/1981 [==============================] - 6s 

Epoch 98/100
1981/1981 [==============================] - 7s 3ms/step - loss: 0.2954 - acc: 0.8778 - val_loss: 0.2276 - val_acc: 0.9082
Epoch 99/100
1981/1981 [==============================] - 6s 3ms/step - loss: 0.2924 - acc: 0.8788 - val_loss: 0.2275 - val_acc: 0.9071
Epoch 100/100
1981/1981 [==============================] - 6s 3ms/step - loss: 0.2913 - acc: 0.8788 - val_loss: 0.2274 - val_acc: 0.9106
Test accuracy: 0.910588234873379
Train on 1981 samples, validate on 850 samples
Epoch 1/150
1981/1981 [==============================] - 8s 4ms/step - loss: 0.6063 - acc: 0.6562 - val_loss: 0.5377 - val_acc: 0.7000
Epoch 2/150
1981/1981 [==============================] - 6s 3ms/step - loss: 0.4956 - acc: 0.8046 - val_loss: 0.4246 - val_acc: 0.8412
Epoch 3/150
1981/1981 [==============================] - 5s 3ms/step - loss: 0.4009 - acc: 0.8501 - val_loss: 0.3391 - val_acc: 0.8776
Epoch 4/150
1981/1981 [==============================] - 6s 3ms/step - loss: 0.3681 - acc: 0.8582 - val_lo

1981/1981 [==============================] - 6s 3ms/step - loss: 0.2708 - acc: 0.8864 - val_loss: 0.2148 - val_acc: 0.9141
Epoch 57/150
1981/1981 [==============================] - 6s 3ms/step - loss: 0.2711 - acc: 0.8864 - val_loss: 0.2146 - val_acc: 0.9141
Epoch 58/150
1981/1981 [==============================] - 6s 3ms/step - loss: 0.2739 - acc: 0.8849 - val_loss: 0.2144 - val_acc: 0.9141
Epoch 59/150
1981/1981 [==============================] - 6s 3ms/step - loss: 0.2753 - acc: 0.8839 - val_loss: 0.2145 - val_acc: 0.9141
Epoch 60/150
1981/1981 [==============================] - 6s 3ms/step - loss: 0.2760 - acc: 0.8854 - val_loss: 0.2143 - val_acc: 0.9153
Epoch 61/150
1981/1981 [==============================] - 6s 3ms/step - loss: 0.2703 - acc: 0.8849 - val_loss: 0.2143 - val_acc: 0.9141
Epoch 62/150
1981/1981 [==============================] - 6s 3ms/step - loss: 0.2716 - acc: 0.8824 - val_loss: 0.2144 - val_acc: 0.9153
Epoch 63/150
1981/1981 [==============================] - 6s 

1981/1981 [==============================] - 6s 3ms/step - loss: 0.2732 - acc: 0.8839 - val_loss: 0.2126 - val_acc: 0.9165
Epoch 117/150
1981/1981 [==============================] - 6s 3ms/step - loss: 0.2731 - acc: 0.8849 - val_loss: 0.2127 - val_acc: 0.9165
Epoch 118/150
1981/1981 [==============================] - 6s 3ms/step - loss: 0.2676 - acc: 0.8859 - val_loss: 0.2127 - val_acc: 0.9165
Epoch 119/150
1981/1981 [==============================] - 6s 3ms/step - loss: 0.2683 - acc: 0.8849 - val_loss: 0.2127 - val_acc: 0.9165
Epoch 120/150
1981/1981 [==============================] - 6s 3ms/step - loss: 0.2739 - acc: 0.8889 - val_loss: 0.2125 - val_acc: 0.9165
Epoch 121/150
1981/1981 [==============================] - 6s 3ms/step - loss: 0.2705 - acc: 0.8859 - val_loss: 0.2125 - val_acc: 0.9165
Epoch 122/150
1981/1981 [==============================] - 6s 3ms/step - loss: 0.2698 - acc: 0.8854 - val_loss: 0.2126 - val_acc: 0.9165
Epoch 123/150
1981/1981 [==============================

...

1981/1981 [==============================] - 5s 3ms/step - loss: 0.3205 - acc: 0.8677 - val_loss: 0.2561 - val_acc: 0.8965
Epoch 65/100
1981/1981 [==============================] - 5s 3ms/step - loss: 0.3131 - acc: 0.8647 - val_loss: 0.2573 - val_acc: 0.9059
Epoch 66/100
1981/1981 [==============================] - 5s 3ms/step - loss: 0.3144 - acc: 0.8728 - val_loss: 0.2584 - val_acc: 0.9106
Epoch 67/100
1981/1981 [==============================] - 5s 3ms/step - loss: 0.3096 - acc: 0.8682 - val_loss: 0.2551 - val_acc: 0.9059
Epoch 68/100
1981/1981 [==============================] - 5s 3ms/step - loss: 0.3070 - acc: 0.8753 - val_loss: 0.2560 - val_acc: 0.9059
Epoch 69/100
1981/1981 [==============================] - 5s 3ms/step - loss: 0.3102 - acc: 0.8733 - val_loss: 0.2552 - val_acc: 0.9035
Epoch 70/100
1981/1981 [==============================] - 5s 3ms/step - loss: 0.3189 - acc: 0.8672 - val_loss: 0.2539 - val_acc: 0.9047
Epoch 71/100
1981/1981 [==============================] - 5s 

1981/1981 [==============================] - 5s 3ms/step - loss: 0.2847 - acc: 0.8839 - val_loss: 0.2186 - val_acc: 0.9165
Epoch 24/200
1981/1981 [==============================] - 5s 3ms/step - loss: 0.2860 - acc: 0.8804 - val_loss: 0.2168 - val_acc: 0.9165
Epoch 25/200
1981/1981 [==============================] - 5s 3ms/step - loss: 0.2847 - acc: 0.8834 - val_loss: 0.2180 - val_acc: 0.9188
Epoch 26/200
1981/1981 [==============================] - 5s 3ms/step - loss: 0.2772 - acc: 0.8834 - val_loss: 0.2155 - val_acc: 0.9165
Epoch 27/200
1981/1981 [==============================] - 5s 3ms/step - loss: 0.2818 - acc: 0.8799 - val_loss: 0.2148 - val_acc: 0.9224
Epoch 28/200
1981/1981 [==============================] - 5s 3ms/step - loss: 0.2884 - acc: 0.8834 - val_loss: 0.2167 - val_acc: 0.9176
Epoch 29/200
1981/1981 [==============================] - 5s 3ms/step - loss: 0.2820 - acc: 0.8788 - val_loss: 0.2172 - val_acc: 0.9165
Epoch 30/200
1981/1981 [==============================] - 5s 

1981/1981 [==============================] - 5s 3ms/step - loss: 0.2814 - acc: 0.8834 - val_loss: 0.2165 - val_acc: 0.9153
Epoch 83/200
1981/1981 [==============================] - 5s 3ms/step - loss: 0.2802 - acc: 0.8839 - val_loss: 0.2165 - val_acc: 0.9141
Epoch 84/200
1981/1981 [==============================] - 5s 3ms/step - loss: 0.2829 - acc: 0.8859 - val_loss: 0.2166 - val_acc: 0.9153
Epoch 85/200
1981/1981 [==============================] - 5s 3ms/step - loss: 0.2810 - acc: 0.8834 - val_loss: 0.2165 - val_acc: 0.9153
Epoch 86/200
1981/1981 [==============================] - 5s 3ms/step - loss: 0.2879 - acc: 0.8879 - val_loss: 0.2164 - val_acc: 0.9141
Epoch 87/200
1981/1981 [==============================] - 5s 3ms/step - loss: 0.2799 - acc: 0.8864 - val_loss: 0.2168 - val_acc: 0.9141
Epoch 88/200
1981/1981 [==============================] - 5s 3ms/step - loss: 0.2831 - acc: 0.8869 - val_loss: 0.2169 - val_acc: 0.9141
Epoch 89/200
1981/1981 [==============================] - 5s 

1981/1981 [==============================] - 5s 3ms/step - loss: 0.2783 - acc: 0.8854 - val_loss: 0.2172 - val_acc: 0.9129
Epoch 143/200
1981/1981 [==============================] - 5s 3ms/step - loss: 0.2780 - acc: 0.8884 - val_loss: 0.2169 - val_acc: 0.9129
Epoch 144/200
1981/1981 [==============================] - 5s 3ms/step - loss: 0.2810 - acc: 0.8849 - val_loss: 0.2171 - val_acc: 0.9129
Epoch 145/200
1981/1981 [==============================] - 5s 3ms/step - loss: 0.2776 - acc: 0.8844 - val_loss: 0.2169 - val_acc: 0.9129
Epoch 146/200
1981/1981 [==============================] - 5s 3ms/step - loss: 0.2782 - acc: 0.8794 - val_loss: 0.2170 - val_acc: 0.9129
Epoch 147/200
1981/1981 [==============================] - 5s 3ms/step - loss: 0.2885 - acc: 0.8884 - val_loss: 0.2171 - val_acc: 0.9129
Epoch 148/200
1981/1981 [==============================] - 5s 3ms/step - loss: 0.2755 - acc: 0.8864 - val_loss: 0.2171 - val_acc: 0.9129
Epoch 149/200
1981/1981 [==============================

1981/1981 [==============================] - 6s 3ms/step - loss: 0.6345 - acc: 0.5982 - val_loss: 0.5893 - val_acc: 0.6376
Epoch 2/50
1981/1981 [==============================] - 5s 3ms/step - loss: 0.5813 - acc: 0.6593 - val_loss: 0.5281 - val_acc: 0.6906
Epoch 3/50
1981/1981 [==============================] - 5s 2ms/step - loss: 0.5046 - acc: 0.7264 - val_loss: 0.4449 - val_acc: 0.7882
Epoch 4/50
1981/1981 [==============================] - 5s 2ms/step - loss: 0.4443 - acc: 0.8258 - val_loss: 0.4079 - val_acc: 0.9082
Epoch 5/50
1981/1981 [==============================] - 5s 2ms/step - loss: 0.4292 - acc: 0.8551 - val_loss: 0.3616 - val_acc: 0.9106
Epoch 6/50
1981/1981 [==============================] - 5s 2ms/step - loss: 0.4032 - acc: 0.8566 - val_loss: 0.3400 - val_acc: 0.9094
Epoch 7/50
1981/1981 [==============================] - 5s 2ms/step - loss: 0.3792 - acc: 0.8627 - val_loss: 0.3012 - val_acc: 0.8941
Epoch 8/50
1981/1981 [==============================] - 5s 2ms/step - los

In [3]:
0.866

0.866

In [4]:
0.817

0.817